# 1. Create PGD Adversarial Flickr Dataset

In [1]:
def attack(pretrained_model, image, epsilon = 0.1):
    adv_img = projected_gradient_descent(pretrained_model, image, epsilon, 
                                                              eps_iter = 0.01, nb_iter = 40, norm = np.inf)
    # true label
    outputs1 = pretrained_model(image.cuda())
    _, preds1 = torch.max(outputs1, 1)
    key1 = str(preds1.detach().cpu().numpy()[0])
    
    # adversarial label
    outputs2 = pretrained_model(adv_img.cuda())
    _, preds2 = torch.max(outputs2, 1)
    key2 = str(preds2.detach().cpu().numpy()[0])
    
    return adv_img[0]

In [2]:
#!mkdir ./data/pgdAttackFlickr

In [3]:
## generate flickr adversarial examples
from torchvision.utils import save_image
from torchvision import datasets, models, transforms
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim 
from PIL import Image
import torch
import torchvision
from torchvision import transforms
import numpy as np

from cleverhans.torch.attacks.projected_gradient_descent import (
    projected_gradient_descent,
)

import glob

path = './data/Images'
subset = glob.glob(path+'/*')
all_images = []
pretrained_model = models.resnet101(pretrained=True).cuda()
pretrained_model.eval()

preprocess = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# count = 0
# total = 0
# for img in tqdm(subset):
#     name = img.split('/')[3]
#     image = Image.open(img).convert('RGB')
#     image = preprocess(image)
#     result = torch.unsqueeze(image, 0)
#     adv_img = attack(pretrained_model, result.cuda())
#     save_image(adv_img, './data/pgdAttackFlickr/'+name)

# 2. Create FGSM Adversarial Flickr Dataset

In [4]:
def attack(pretrained_model, image, epsilon = 0.1):
    adv_img = fast_gradient_method(pretrained_model, image, epsilon, norm = np.inf)
    # true label
    outputs1 = pretrained_model(image.cuda())
    _, preds1 = torch.max(outputs1, 1)
    key1 = str(preds1.detach().cpu().numpy()[0])
    
    # adversarial label
    outputs2 = pretrained_model(adv_img.cuda())
    _, preds2 = torch.max(outputs2, 1)
    key2 = str(preds2.detach().cpu().numpy()[0])
    
    return adv_img[0]

In [5]:
# !mkdir ./data/fgsmAttackFlickr

In [6]:
## generate flickr adversarial examples
from torchvision.utils import save_image
from torchvision import datasets, models, transforms
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim 
from PIL import Image
import torch
import torchvision
from torchvision import transforms
import numpy as np

from cleverhans.torch.attacks.fast_gradient_method import (
    fast_gradient_method,
)

import glob

path = './data/Images'
subset = glob.glob(path+'/*')
all_images = []
pretrained_model = models.resnet101(pretrained=True).cuda()
pretrained_model.eval()

preprocess = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# count = 0
# total = 0
# for img in tqdm(subset):
#     name = img.split('/')[3]
#     image = Image.open(img).convert('RGB')
#     image = preprocess(image)
#     result = torch.unsqueeze(image, 0)
#     adv_img = attack(pretrained_model, result.cuda())
#     save_image(adv_img, './data/fgsmAttackFlickr/'+name)

# 3. Finetune Model 

In [7]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class image_title_dataset(Dataset):
    def __init__(self, list_image_path,list_txt, attack):

        self.image_path = list_image_path
        self.title  = clip.tokenize(list_txt) #you can tokenize everything at once in here(slow at the beginning), or tokenize it in the training loop.
        self.attack = attack
        
    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        if(self.attack=="pgd"):
            folder = "pgdAttackFlickr"
        else:
            folder = "fgsmAttackFlickr"
        
        image = Image.open("./data/"+folder+"/"+self.image_path[idx]).convert('RGB')
        image = preprocess(image)
        #image = preprocess(Image.open(self.image_path[idx])) # Image from PIL module
        title = self.title[idx]
        return image,title

In [8]:
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

In [9]:
class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg, self.sum, self.count = [0] * 3

    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

In [10]:
def make_train_valid_dfs():
    dataframe = pd.read_csv("data/captions.txt")
    dataframe.insert(0, 'id', range(0, len(dataframe)))
    max_id = dataframe["id"].max() + 1

    image_ids = np.arange(0, max_id)
    np.random.seed(42)
    valid_ids = np.random.choice(
        image_ids, size=int(0.05 * len(image_ids)), replace=False
    )
    train_ids = [id_ for id_ in image_ids if id_ not in valid_ids]
    train_dataframe = dataframe[dataframe["id"].isin(train_ids)].reset_index(drop=True)
    valid_dataframe = dataframe[dataframe["id"].isin(valid_ids)].reset_index(drop=True)
    return train_dataframe, valid_dataframe


In [11]:
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim 
import numpy as np
import clip
from PIL import Image

def write_logs(file_name, message):
    f = open(file_name, "a")
    f.write(message + '\n')
    f.close()
    
def train(df, lr= 5e-5, noise=None, epochs=10, batch_size=64, attack = None):

    device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
    model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
    if noise is not None:
        preprocess.transforms.append(AddGaussianNoise(0, noise))
    loss_img = nn.CrossEntropyLoss()
    loss_txt = nn.CrossEntropyLoss()
    #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset
    optimizer = optim.Adam(model.parameters(), lr ,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) 
    
    list_image_path = df['image'].values
    list_txt = df['caption'].values
    dataset = image_title_dataset(list_image_path,list_txt, attack)
    train_dataloader = DataLoader(dataset,batch_size = batch_size) #Define your own dataloader
    best_loss = float("inf")
    log_file_name = attack+"_train_logs"
    for epoch in tqdm(range(epochs), total=epochs):
        loss_meter = AvgMeter()
        for batch in train_dataloader:
            optimizer.zero_grad()
            images, texts = batch 
            images = images.to(device)
            texts = texts.to(device)
            logits_per_image, logits_per_text = model(images, texts)
            ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
            total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            total_loss.backward()
            count = images.size(0)
            loss_meter.update(total_loss.item(), count)
            
            if device == "cpu":
                optimizer.step()
            else: 
                convert_models_to_fp32(model)
                optimizer.step()
                clip.model.convert_weights(model)
            
        if loss_meter.avg < best_loss:
            best_loss= loss_meter.avg
            torch.save(model.state_dict(), f"adv_trained_{attack}_best2.pt")
            print("Saved Best Model!")
            
        write_logs(log_file_name, f"{epoch}, {loss_meter.avg:.5f}, {best_loss:.5f}")             
        print(f"Epoch: {epoch + 1}, train_loss: {loss_meter.avg:.5f}, best_loss: {best_loss:.5f}")

In [12]:
train_dataframe, valid_dataframe = make_train_valid_dfs()
print(valid_dataframe.shape)


for att in ["pgd", "fgsm"]:
    print(f"Adversarial training on {att} dataset")
    train(valid_dataframe, 5e-5, epochs= 10, attack = att)

(2022, 3)
Adversarial training on pgd dataset


 10%|█████▌                                                 | 1/10 [00:16<02:31, 16.82s/it]

Saved Best Model!
Epoch: 1, train_loss: 1.72762, best_loss: 1.72762


 20%|███████████                                            | 2/10 [00:33<02:16, 17.00s/it]

Saved Best Model!
Epoch: 2, train_loss: 0.68656, best_loss: 0.68656


...

Epoch: 8, train_loss: 1.12028, best_loss: 0.61778


 90%|█████████████████████████████████████████████████▌     | 9/10 [02:17<00:14, 14.62s/it]

Epoch: 9, train_loss: 1.05675, best_loss: 0.61778


100%|██████████████████████████████████████████████████████| 10/10 [02:31<00:00, 15.18s/it]

Epoch: 10, train_loss: 1.19895, best_loss: 0.61778
